In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import mysql.connector
import psycopg2
from sqlalchemy import create_engine
from sqlalchemy import URL

In [4]:
from config import mysql_qm 
from config import postgresql_home 

In [8]:
sql_query = '''
    SELECT
        *
    FROM
        sales 
        INNER JOIN companies USING(Артель)
        INNER JOIN chaebols USING(Артель)
        INNER JOIN industries USING(Товар) 
        INNER JOIN regions USING(Град)
'''

In [9]:
with mysql.connector.connect(
            host = mysql_qm['host'],
            user = mysql_qm['user'], 
            password = mysql_qm['password'],
            database = 'kvadromash_delete'
            ) as connection:
    df = pd.read_sql(sql_query, connection)

C:\Users\Боря\AppData\Local\Temp\ipykernel_6584\2949249702.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_query, connection)


In [12]:
df = df[['Дата', 'Плата', 'Артель', 'Чеболь', 'Товар', 'Промысел', 'Град', 'Царство', ]]

In [14]:
df.Дата = pd.to_datetime(df.Дата)

In [16]:
df.head()

,Дата,Плата,Артель,Чеболь,Товар,Промысел,Град,Царство
0,2048-01-01,10208.64,Каша из топора,Дуремар,веган,яства,Пушград,Лукоморье
1,2048-01-01,21961.11,Бабки Ёжки,Синдбад,порошки,зелья,Пушград,Лукоморье
2,2048-01-01,6905.58,Хоббитонские угощения,Синдбад,сласти,яства,Пушград,Лукоморье
3,2048-01-01,4571.70,Дырявый котёл,Дуремар,киселя,яства,Лукоморск,Лукоморье
4,2048-01-01,15124.69,Ёлки-палки,Синдбад,палочки,артефакторика,Тумантархан,Тмутаракань


In [18]:
cube = df.groupby(['Чеболь', 'Промысел', 'Царство', ]).Плата.sum().round(2).reset_index()

In [20]:
cube = cube.rename(columns={'Плата': 'Сумма_продаж'})

In [22]:
cube.head()

,Чеболь,Промысел,Царство,Сумма_продаж
0,Дуремар,артефакторика,Лукоморье,54196.00
1,Дуремар,артефакторика,Нарния,126804.89
2,Дуремар,артефакторика,Тридесятое,110733.37
3,Дуремар,всячина,Лукоморье,74514.85
4,Дуремар,всячина,Нарния,24891.29


запишем полученную таблицу в БД Postgres

In [24]:
sql_create_query = '''
    CREATE TABLE IF NOT EXISTS cube (
        Чеболь       VARCHAR(64), 
        Промысел     VARCHAR(64), 
        Царство      VARCHAR(64), 
        Сумма_продаж DECIMAL(10, 2)
    )
'''

In [26]:
with psycopg2.connect(
            host=postgresql_home['host'],
            user=postgresql_home['user'], 
            password=postgresql_home['password'],
            database='competition_analysis'
            ) as connection:
    with connection.cursor() as cursor:
        cursor.execute(sql_create_query)

In [28]:
url_object = URL.create(
    "postgresql+psycopg2",
    username=postgresql_home['user'],
    password=postgresql_home['password'], 
    host=postgresql_home['host'],
    port=postgresql_home['port'],
    database='competition_analysis',
)

In [30]:
engine = create_engine(url_object)

In [32]:
cube.to_sql(
    name='cube', # имя таблицы
    con=engine,  # движок
    if_exists="replace", # если в таблице данные уже есть, заменяем их
    index=False # без индекса датафрейма
)

42

In [ ]:
companies_by_months = sales.pivot_table('Плата', 'Артель', sales.Дата.dt.to_period('M'), aggfunc='sum').reset_index()
companies_by_months.head()
companies_by_months.to_csv('companies_by_months.csv', index=False)

Проблема с мерцанием экрана: Settings Editor → Notebook → Windowing mode - Defer